In [3]:
import requests, json, sys
import pandas as pd

sys.path.insert(1, '../../../scripts/')
from s3_support import *

In [10]:
# Constants

years = [i for i in range(207, 2020)]

In [4]:
# Fetch a list of our clients and their Tax IDs

In [6]:
# Fetch list of our organizations
organizations = get_dataframe_from_file("qgiv-stats-data", "organizations.csv")

# Reduce the dataset to only the columns we need
organizations = organizations[['Id', 'Org Name', 'Tax ID', 'Go Live Date']]

In [ ]:
# Helpers needed to parse open_data

In [ ]:
from lxml import etree
from lxml import etree

def xml_to_dict(xml_str):
    return etree2dict(etree.fromstring(xml_str))

# converts an etree to dict, useful to convert xml to dict
def etree2dict(tree):
    root, contents = recursive_dict(tree)
    return {root: contents}

def recursive_dict(element):
    if element.attrib and 'type' in element.attrib and element.attrib['type'] == "array":
        return element.tag.replace('{http://www.irs.gov/efile}', ''), [(dict(map(recursive_dict, child)) or getElementValue(child)) for child in element]
    else:
        return element.tag.replace('{http://www.irs.gov/efile}', ''), dict(map(recursive_dict, element)) or getElementValue(element)

def getElementValue(element):
    if element.text:
        if element.attrib and 'type' in element.attrib:
            attr_type = element.attrib.get('type')
            if attr_type == 'integer':
                return int(element.text.strip())
            if attr_type == 'float':
                return float(element.text.strip())
            if attr_type == 'boolean':
                return element.text.lower().strip() == 'true'
            if attr_type == 'datetime':
                return element.text.strip()
        else:
            return element.text
    elif element.attrib:
        if 'nil' in element.attrib:
            return None
        else:
            return element.attrib
    else:
        return None

In [18]:
def open_data_revenue(tax_id: str, year: int) -> int:
    url = 'https://s3.amazonaws.com/irs-form-990/index_{}.csv'

    # The year index contains the identifier we need in order to fetch the revenue data.
    year_index = pd.read_csv(url.format(year), error_bad_lines=False)
    year_index['EIN'] = year_index['EIN'].astype(str)

    # Get the identifier from the year index
    index = year_index[year_index['EIN'] == tax_id]
    if len(index.values) == 0: return None
        
    identifier = index['OBJECT_ID'].values[0]

    try:
        response = requests.get("https://s3.amazonaws.com/irs-form-990/{}_public.xml".format(identifier))
        tax_info = xml_to_dict(response.content)

        return tax_info['Return']['ReturnData']['IRS990']['Revenue']

    except:
        print('Error requesting tax information for {} in year {}'.format(tax_id, year)
        return None

print(open_data_revenue(tax_id='060693092', year=2015))


SyntaxError: invalid syntax (<ipython-input-18-d1da9f8719f0>, line 22)

In [7]:
def fetch_revenue(tax_id: int, year: int) -> int:
    pass



,Id,Org Name,Tax ID,Go Live Date
0,441696,NaN,NaN,12/31/1969
1,1045,FROM Food Fight,NaN,03/14/2011
2,442134,Camp Laurelwood,060693092,11/06/2017
3,442293,Cass Community Social Services,383429921,01/29/2018
4,14274,CFSSWMI,NaN,12/07/2012
...,...,...,...,...
5206,54276,Zendesk Test,NaN,12/31/1969
5207,442509,Zeno,205570858,12/31/1969
5208,192,Zero Breast Cancer,NaN,08/22/2008
5209,442543,Zion Lutheran Church,846033364,12/31/1969
